In [1]:
import ee
import geemap

In [2]:
# ee.Authenticate()

In [3]:
ee.Initialize()

In [4]:
Map = geemap.Map()

In [5]:
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [6]:
#*** Start of imports. If edited, may not auto-convert in the playground. ***#
Np = ee.FeatureCollection("users/aaditya-rb/nepal_map")
Kaski = ee.FeatureCollection("users/aaditya-rb/kaski_test")
LULC_icimod = ee.Image("users/aaditya-rb/np_lc_2010_v2f")
L7 = ee.ImageCollection("LANDSAT/LE07/C01/T1_SR")
DEM = ee.Image("USGS/SRTMGL1_003")
L7_vis = {"opacity":1,"bands":["B4","B3","B2"],"min":220,"max":19660,"gamma":1}
LULC_vis = {"opacity":1,"bands":["b1"],"palette":["0cff35","ff0000","1202ff","fbff08","ff8792","b6ffb4","c0bcff","b4e0ff","ffc38f","feb3ff","6679ff","5c69ff"]}
#**** End of imports. If edited, may not auto-convert in the playground. ****#

In [7]:
Map.setCenter(84.1240, 28.3949, 7)
L7_np = L7.filterDate('2010-01-01', '2010-12-31') \
.filterBounds(Np) \
.filterMetadata("CLOUD_COVER", 'less_than', 10) \
.sort('CLOUD_COVER',False)
# print(L7_np.size())

In [8]:
mergedL7 = L7_np.median().clip(Np)

# Map.addLayer(mergedL7, L7_vis, "Landsat 7")
# Map.addLayer(LULC_icimod, LULC_vis, "LULC icimod")

b2 = mergedL7.select("B2")
b3 = mergedL7.select("B3")
b4 = mergedL7.select("B4")
b5 = mergedL7.select("B5")
NDVI = b4.subtract(b3).divide(b4.add(b3)).rename("NDVI")
NDBI = b5.subtract(b4).divide(b5.add(b4)).rename("NDBI")
NDWI = b4.subtract(b5).divide(b5.add(b4)).rename("NDWI")
MNDWI = b2.subtract(b5).divide(b2.add(b5)).rename("MNDWI")
DEM_nepal = DEM.clip(Np)


In [9]:
addedNepal = mergedL7 \
# .addBands(NDVI) \
# .addBands(NDBI) \
# .addBands(NDWI) \
# .addBands(MNDWI) \
# .addBands(LULC_icimod.rename("LULC"))


In [10]:
Map.addLayer(addedNepal, L7_vis, "Landsat 7")
randPoints = ee.FeatureCollection.randomPoints(Np, 100000, 2)
Map.addLayer(randPoints, {}, "Random Points")

In [11]:
Map

Map(center=[28.3949, 84.124], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(chil…

In [12]:
exportBands = mergedL7.select("B1").addBands(mergedL7.select("B2")) \
.addBands(mergedL7.select("B3")) \
.addBands(mergedL7.select("B4")) \
.addBands(mergedL7.select("B5")) \
.addBands(mergedL7.select("B6")) \
.addBands(mergedL7.select("B7")) \
.addBands(NDVI.select("NDVI")) \
.addBands(NDBI.select("NDBI")) \
.addBands(NDWI.select("NDWI"))\
.addBands(MNDWI.select("MNDWI"))\
.addBands(DEM_nepal) \
.addBands(LULC_icimod.select("b1").rename("LULC"))


In [14]:
out_csv = 'landsatDataNP_final.csv'
geemap.extract_values_to_points(randPoints, exportBands, out_csv, scale = 30)

Generating URL ...
Please wait ...
Data downloaded to D:\GIS\LULC_GEE_ICIMOD\use_files\landsatDataNP_final.csv
